# Projeto 2 - Ciência dos Dados

Nome: Antonio Saporiti

Nome: Enzo Neto

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [2]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @Enzo56212670

#leitura do arquivo no formato JSON
with open('auth ELLEN.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [5]:
a=[1,2,3]
b=[22,3,4]
a+b

[1, 2, 3, 22, 3, 4]

In [6]:
p1=pd.DataFrame(a)
p1

,0
0,1
1,2
2,3


In [7]:
list(p1[0])

[1, 2, 3]

In [8]:
#Produto escolhido:
produto = 'Rainbow six siege'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'en'

Capturando os dados do twitter:

In [9]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [10]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [11]:
r6_raw = pd.read_excel("Rainbow six siege.xlsx")
r6_raw

,Treinamento
0,rt @infuprise: launching 50%... 🚀\n\nwe are lo...
1,rainbow six siege - in depth: new morocco dlc ...
2,rt @streamsupport5: it's been fun watching @tk...
3,rt @rainbow6game: we're excited to announce th...
4,trickstar_wv is #streaming tom clancy's rainbo...
5,rt @rainbow6game: it’s your last chance to sav...
6,who is 「rainbow six siege」 leader...?
7,tom clancy's rainbow six siege / south park : ...
8,check out tom clancy's rainbow six: siege wors...
9,@ubisoft @rainbow6game .... yeah. we just need...


In [12]:
r6 = set(r6_raw.Treinamento)
r6

{'"#9874hrs clan" rainbow six siege https://t.co/ucikpo9edv',
 '#breadisoverparty bread stole my ace on rainbow six siege',
 "#domino427 is now streaming #tom clancy's rainbow six: siege https://t.co/oiq6wvwnjs",
 "#friendlyfps is now streaming #tom clancy's rainbow six: siege https://t.co/whqsbw3fpr",
 "#gameon thezoroark31 transmitiendo tom clancy's rainbow six: siege pasen a ver https://t.co/ividpolt0r https://t.co/h00smzb5o0",
 "#kinggeorge is now streaming #tom clancy's rainbow six: siege https://t.co/whi68icd2c",
 "#lilredhydra is now streaming #tom clancy's rainbow six: siege https://t.co/j5dqlh6lux",
 "#maximumcaliber is now streaming #tom clancy's rainbow six: siege https://t.co/sj3prrxgox",
 '#naruto rainbow six siege 3d print t shirt https://t.co/q6biqzqqmn https://t.co/ntmbb53kks',
 '#rainbowsixsiege #rss #rainbowsixseige #rainbowsix - \nrainbow six siege: patch notes 4.3 – ember rise update https://t.co/w9fpmwuuxl',
 "#skaman1978 is now streaming #tom clancy's rainbow six:

In [13]:
r6_lista = pd.Series(list(r6))
r6_lista.to_excel("Treinamento_r6.xlsx")

In [14]:
treinamento_r6 = pd.read_excel("Treinamento_r6.xlsx")
treinamento_r6_2 = pd.read_excel("Treinamento_r6.1.xlsx")
treinamento_r6

,0
0,just a teaser of things to come:\nrainbow six ...
1,rt @k_m_p_gamer: check out tom clancy's rainbo...
2,"hey #familyguise i am currently live, we are g..."
3,god of war\nuncharted 4: a thief’s end\nbloodb...
4,my next rainbow six siege montage gonna be lit...
5,@ubisoft hey ubisoft i been playing rainbow si...
6,@i_am_wildcat rainbow six siege is the same
7,trickstar_wv is #streaming tom clancy's rainbo...
8,rt @itsenyoo: how to not play rainbow six sieg...
9,check out my broadcast from my playstation 4! ...


Somando as tabelas para criação da tabela Treinamento com 300 tweets

In [15]:
a = treinamento_r6.iloc[:150]
b = treinamento_r6_2.iloc[:150]
listaT1 = list(a[0])
listaT2 = list(b["tweets"])
Treinamento_300 = listaT1 + listaT2

In [16]:
treinamento_300 = pd.Series(Treinamento_300)
treinamento_300.to_excel("Treinamento300.xlsx")

In [23]:
treinamento_300 = pd.read_excel("Treinamento300.feita.xlsx")
treinamento_300

,tweets,Categoria
0,i'm now streaming on twitch! playing tom clanc...,1
1,rt @imebyte: new video!\n\nrecoil control cons...,1
2,rainbow six siege #126\nhttps://t.co/lyszfpexb...,1
3,#breadisoverparty bread stole my ace on rainbo...,1
4,"@i_am_wildcat well, rainbow six siege is a goo...",1
5,@ubisoft hey ubisoft i been playing rainbow si...,1
6,does anyone know when new rainbow six siege se...,1
7,"nobody:\nrainbow six siege players: ""oh this n...",0
8,who is 「rainbow six siege」 leader...?,0
9,@cherryrae @osamadorias rainbow six siege's gr...,1


No quesito categoria, foram definidos que o valor 1 representa "gosta" (do jogo) e 0 representa "não gosta" (do jogo).

OBS: ao final do trabalho, o jupyter foi rodado e novas tabelas foram criadas. Felizmente, tinhamos a outra tabela já feita por meio de manipulações no jupyter salva antes do mesmo ser rodado e por isso há uma tabela Treinamento300 (que é a tabela que o jupyter montou ao rodar o programa pela ultima vez) e a tabela Treinamento300.feita (que foi a tabela feita antes do Jupyter ser rodado pela última vez, já classificada por nós).

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [18]:
import re 


def cleanup(text):

    import string
    punctuation = '[!-.:?;]'
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [24]:
r6_gosta = treinamento_300[treinamento_300.Categoria == 1]
r6_nao_gosta = treinamento_300[treinamento_300.Categoria == 0]

Começando o processo de cálculo das probabilidades de cada grupo

In [26]:
lista_r6_gosta = pd.Series(list(r6_gosta["tweets"]))
lista_r6_nao_gosta = pd.Series(list(r6_nao_gosta["tweets"]))
lista_r6_gosta

0      i'm now streaming on twitch! playing tom clanc...
1      rt @imebyte: new video!\n\nrecoil control cons...
2      rainbow six siege #126\nhttps://t.co/lyszfpexb...
3      #breadisoverparty bread stole my ace on rainbo...
4      @i_am_wildcat well, rainbow six siege is a goo...
5      @ubisoft hey ubisoft i been playing rainbow si...
6      does anyone know when new rainbow six siege se...
7      @cherryrae @osamadorias rainbow six siege's gr...
8      where is the rainbow six siege ember rise rele...
9      frodo_86 played tom clancy's rainbow six siege...
10             rainbow six siege https://t.co/xysag2wsn9
11     rt @bowskij: rainbow six siege fails &amp; hig...
12     check out my broadcast from my playstation 4! ...
13             rainbow six siege https://t.co/uvrd5kdxo2
14     new rainbow six siege montage come check it ou...
15     rt @seahawk101bl: just posted a video 2v1 you'...
16     rainbow six siege buffs fuze and warden for em...
17     rainbow six siege casual

In [35]:
gosta = []
for i in lista_r6_gosta:
    gosta.append(i)
gosta_texto=', '.join(gosta)

with open('gosta.txt','w', encoding="utf8") as arquivo:
    arquivo.write(gosta_texto)

In [34]:
n_gosta = []
for i in lista_r6_nao_gosta:
    gosta.append(i)
gosta_texto=', '.join(gosta)

with open('n_gosta.txt','w', encoding="utf8") as arquivo:
    arquivo.write(gosta_texto)

In [36]:
like_r6_sujo = open("gosta.txt", "r", encoding="utf8").read()
dislike_r6_sujo = open("n_gosta.txt", "r", encoding="utf8").read()

In [38]:
like_r6 = cleanup(like_r6_sujo.lower())
dislike_r6 = cleanup(dislike_r6_sujo.lower())

In [39]:
print(like_r6[0:100])
print(dislike_r6[0:100])

i m now streaming on twitch  playing tom clancy s rainbow six  siege https //t co/3lld7mrnqk  rt @im
i m now streaming on twitch  playing tom clancy s rainbow six  siege https //t co/3lld7mrnqk  rt @im


In [40]:
serie_like = pd.Series(like_r6.slpit())
serie_dislike = pd.Series(dislike_r6.split())

AttributeError: 'str' object has no attribute 'slpit'

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

Código do professor dado em aula para classificar uma frase, caso pertencesse à Camôes ou à Lusíadas.